In [1]:
import pennylane as qml
import tensorflow as tf
import PennyLane_Gate_Configurations as pgc

In [2]:
num_of_qubits = input("Enter number of qubits (default=4): ")
if num_of_qubits == "":
    num_of_qubits = 4
else:
    try:
        num_of_qubits = int(num_of_qubits)
    except Exception as error:
        print(error)
        
weight_shapes = {"weights": num_of_qubits}

In [ ]:
dev = qml.device('default.qubit', wires=num_of_qubits, shots=1000)

In [4]:
@qml.qnode(dev)
def qnode(inputs, weights): 
    # Data Encoding with the inputs
    pgc.data_encoding(inputs)
    
    # Constructing the rest of the circuit with the weights
    pgc.cx_almost_all_neighbors(num_of_qubits)
    pgc.build_cascade_ry(weights, num_of_qubits)
    
    # Return measurement layer output
    return tuple([qml.expval(qml.PauliZ(i)) for i in range(num_of_qubits)])

In [5]:
# Verification only
if num_of_qubits == 4:
    inp = [0, 0.1, 1.5, 0.33]
    wts = [0.25, 0.33, 0.75, 0.67]

    print(qml.draw(qnode)(inputs=inp, weights=wts))

0: ──RX(0.00)─╭●──RY(0.25)─────────────────────┤  <Z>
1: ──RX(0.10)─╰X─╭●─────────RY(0.33)───────────┤  <Z>
2: ──RX(1.50)────╰X────────╭●─────────RY(0.75)─┤  <Z>
3: ──RX(0.33)──────────────╰X─────────RY(0.67)─┤  <Z>


In [6]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=num_of_qubits, dtype='float64')

In [7]:
# Verification only
# qlayer.get_config()

In [8]:
# Verification only
if num_of_qubits == 4:
    t = qlayer.qnode(inp, wts)
    print(t)

tf.Tensor([0.976 0.948 0.094 0.09 ], shape=(4,), dtype=float64)
